In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [3]:
docs

[Document(page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…', metadata={'source': 'speech.txt'}),
 Document(page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high sp

In [4]:
embeddings = OllamaEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db

In [5]:
# querying the DB

query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever = db.as_retriever()
retriever.invoke(query)

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [7]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
  15625.496),
 (Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we hav

In [8]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.8374478816986084,
 -3.0862040519714355,
 1.9563708305358887,
 1.5297901630401611,
 -0.9171241521835327,
 0.6330394148826599,
 1.552098274230957,
 -0.5665127038955688,
 0.9155701994895935,
 -2.01057505607605,
 1.2468286752700806,
 -1.9554243087768555,
 -0.6762589812278748,
 1.5483592748641968,
 -0.5183047652244568,
 -1.9872207641601562,
 -0.37298980355262756,
 0.062495335936546326,
 0.6922306418418884,
 -1.8537113666534424,
 -0.6913982033729553,
 -1.0273534059524536,
 2.147757053375244,
 -1.9758224487304688,
 0.7530924081802368,
 -0.9805992841720581,
 0.395979106426239,
 -1.6467844247817993,
 0.13645413517951965,
 -0.8596445322036743,
 2.122737169265747,
 -2.8618502616882324,
 -2.7179512977600098,
 3.7435684204101562,
 2.327260732650757,
 -5.075565338134766,
 -1.0977346897125244,
 1.3929725885391235,
 -1.222432017326355,
 -1.3927972316741943,
 -0.5087866187095642,
 -1.997821569442749,
 2.195685863494873,
 0.35554268956184387,
 0.9504433274269104,
 0.5525370240211487,
 -0.254867285490

In [9]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid

In [10]:
### Saving And Loading
db.save_local("faiss_index")

In [11]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [12]:
docs

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid